In [1]:
import tensorflow as tf
import numpy as np
import random
sess = tf.InteractiveSession()

In [2]:
names = np.loadtxt('/tmp/Practical_RL/week7_[recap]_rnn/names', dtype=str, delimiter=';').tolist()

In [3]:
random.shuffle(names)

In [4]:
names[:5]

['Alvinia', 'Rolland', 'Joanna', 'Susanne', 'Sarajane']

In [5]:
len(names)

7944

In [6]:
chars = set(''.join(names)+'_')
dic = {x:i for i,x in enumerate(chars)}

In [7]:
def convert_to_ix(names):
    if type(names[0]) != str:
        names = list(map(lambda x: x.decode('utf-8'), names))
    chars = []
    max_len = max([len(x) for x in names])
    filler = dic['_']
    for name in names:
        chars.append([])
        for s in name:
            chars[-1].append(dic[s])
        chars[-1] += [filler]*(max_len-len(name))
#     print(chars)
    return np.array(chars)

In [8]:
convert_to_ix(names[:3])

array([[32, 24, 41, 52, 31, 52, 28],
       [ 8, 18, 24, 24, 28, 31, 15],
       [16, 18, 28, 31, 31, 28,  0]])

In [9]:
def get_random_states(cell, batch_size):
    state_variables=[]
    for state_c, state_h in cell.zero_state(batch_size, tf.float32):
            state_variables.append(tf.contrib.rnn.LSTMStateTuple(
                tf.random_normal(tf.shape(state_c), stddev=0.01),
                tf.random_normal(tf.shape(state_h), stddev=0.01)))
    return tuple(state_variables)

In [10]:
def process_names(names):
    with tf.variable_scope('test', reuse=tf.AUTO_REUSE):
        inp = names

        forward_lookup = tf.py_func(convert_to_ix, [inp], tf.int64)
        forward_lookup = tf.nn.embedding_lookup(tf.diag(tf.range(len(chars))), forward_lookup)
        forward_lookup = tf.reshape(forward_lookup, [tf.shape(inp)[0], -1, len(chars)])
        forward_lookup = tf.cast(forward_lookup, tf.float32)

        batch_size, series_len = tf.shape(forward_lookup)[0], tf.shape(forward_lookup)[1]

        cells = [tf.nn.rnn_cell.LSTMCell(128, name='cell_{}'.format(i), activation=tf.nn.tanh) for i in range(3)]
        mul_rnn = tf.nn.rnn_cell.MultiRNNCell(cells)

        flt = tf.transpose(forward_lookup, [1,0,2])
        init_state = get_random_states(mul_rnn, batch_size)#mul_rnn.zero_state(batch_size=batch_size, dtype=tf.float32)
        
        init = (tf.zeros([batch_size, mul_rnn.state_size[0][0]]), init_state)
        out,_ = tf.scan(lambda prev,x: mul_rnn(x, prev[1]), flt, initializer=init)
        out = tf.transpose(out, [1,0,2])
        
        dim = out.shape[-1]
        out = tf.reshape(out, [-1, dim])
        with tf.variable_scope('ww', initializer=tf.random_normal_initializer(stddev=0.01)):
            W = tf.get_variable('W',shape=[dim, len(chars)])
            b = tf.get_variable('b', shape=[1,len(chars)])
        out = tf.matmul(out,W) + b

        out = tf.reshape(out, [batch_size, series_len, len(chars)])

        preds = out[:,:-1]
        targets = forward_lookup[:,1:]
        loss = tf.nn.softmax_cross_entropy_with_logits_v2(labels=targets, logits=preds)
        loss = tf.reduce_mean(loss)
        return out, loss

In [11]:
train_data = tf.Variable(np.array(names), trainable=False, name='train_data')
shuffle_op = tf.assign(train_data, tf.random_shuffle(train_data))
train_crop = tf.random_crop(train_data, [100])

In [12]:
out, loss = process_names(train_crop)

Instructions for updating:
Use the retry module or similar alternatives.


In [13]:
summary = tf.summary.scalar('loss', loss)
!mkdir /tmp/tfdbg
writer = tf.summary.FileWriter('/tmp/tfdbg/2')

mkdir: cannot create directory ‘/tmp/tfdbg’: File exists


In [14]:
opt = tf.train.AdamOptimizer(0.001).minimize(loss)

In [15]:
tf.global_variables_initializer().run()

In [ ]:
for i in range(100000):
    opt.run()
    if i %200 == 0:
        print(loss.eval())
        shuffle_op.eval()
    if i % 20 == 0:
        s = summary.eval()
        writer.add_summary(s)

61.41399
47.69909
44.349922
48.12338
51.0445
47.30741
73.736824
61.064575
70.388306
72.30483
85.21728
80.641205
73.937454
100.21162
92.855316
91.25403


In [13]:
out.eval().shape

(30, 10, 60)

In [38]:
fb = lambda prev,_: [prev[1],prev[0]+prev[1]]

In [39]:
fbs = tf.scan(fb, tf.range(10), initializer=[1.,1.])

In [41]:
fbs[0].eval()

array([ 1.,  2.,  3.,  5.,  8., 13., 21., 34., 55., 89.], dtype=float32)

In [42]:
fbs

[<tf.Tensor 'scan_7/TensorArrayStack/TensorArrayGatherV3:0' shape=(10,) dtype=float32>,
 <tf.Tensor 'scan_7/TensorArrayStack_1/TensorArrayGatherV3:0' shape=(10,) dtype=float32>]

In [12]:
cells = [tf.nn.rnn_cell.BasicLSTMCell(80, name='cell_{}'.format(i)) for i in range(3)]
cell = tf.nn.rnn_cell.MultiRNNCell(cells)

In [13]:
cell.zero_state(5, tf.float32)

(LSTMStateTuple(c=<tf.Tensor 'MultiRNNCellZeroState/BasicLSTMCellZeroState/zeros:0' shape=(5, 80) dtype=float32>, h=<tf.Tensor 'MultiRNNCellZeroState/BasicLSTMCellZeroState/zeros_1:0' shape=(5, 80) dtype=float32>),
 LSTMStateTuple(c=<tf.Tensor 'MultiRNNCellZeroState/BasicLSTMCellZeroState_1/zeros:0' shape=(5, 80) dtype=float32>, h=<tf.Tensor 'MultiRNNCellZeroState/BasicLSTMCellZeroState_1/zeros_1:0' shape=(5, 80) dtype=float32>),
 LSTMStateTuple(c=<tf.Tensor 'MultiRNNCellZeroState/BasicLSTMCellZeroState_2/zeros:0' shape=(5, 80) dtype=float32>, h=<tf.Tensor 'MultiRNNCellZeroState/BasicLSTMCellZeroState_2/zeros_1:0' shape=(5, 80) dtype=float32>))

In [15]:
state_variables

[LSTMStateTuple(c=<tf.Variable 'Variable:0' shape=(5, 80) dtype=float32_ref>, h=<tf.Variable 'Variable_1:0' shape=(5, 80) dtype=float32_ref>),
 LSTMStateTuple(c=<tf.Variable 'Variable_2:0' shape=(5, 80) dtype=float32_ref>, h=<tf.Variable 'Variable_3:0' shape=(5, 80) dtype=float32_ref>),
 LSTMStateTuple(c=<tf.Variable 'Variable_4:0' shape=(5, 80) dtype=float32_ref>, h=<tf.Variable 'Variable_5:0' shape=(5, 80) dtype=float32_ref>)]

In [16]:
state_c

<tf.Tensor 'MultiRNNCellZeroState_1/BasicLSTMCellZeroState_2/zeros:0' shape=(5, 80) dtype=float32>